In [35]:
import pandas as pd
import os
import requests
import json

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [36]:
#Setting the api_key variable
api_key_fsq = os.environ["FOURSQUARE_API_KEY"]

In [37]:
#Reading in the vancouver_bikes dataframe, which includes the locations of all bike stations in Vancouver
vancouver_bikes = pd.read_csv("vancouver_bikes.csv")
vancouver_bikes.head()

,uid,latitude,longitude,free_bikes,empty_slots,station_name
0,1,49.262487,-123.114397,1,32,10th & Cambie
1,4,49.274566,-123.121817,10,5,Yaletown-Roundhouse Station
2,5,49.279764,-123.110154,8,16,Dunsmuir & Beatty
3,7,49.260599,-123.113504,3,12,12th & Yukon (City Hall)
4,8,49.264215,-123.117772,3,12,8th & Ash


In [38]:
#Creating a function to make an API request to Foursquare
def get_venues_fsq(latitude, longitude, radius, api_key, categories):
    url = "https://api.foursquare.com/v3/places/search"
    
    headers = {
        "Accept": "application/json",
        "Authorization": api_key
    }
    
    params = {
        "ll": f"{latitude},{longitude}",
        "radius": radius,
        "categories": categories,
        "limit": 50
    }
    
    response = requests.get(url, headers=headers, params=params)
    return response

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [39]:
#Test request for the first row of vancouver_bikes df
res = get_venues_fsq(latitude = 49.262487, 
                      longitude = -123.114397, 
                      radius = 1000, 
                      api_key =  api_key_fsq, 
                      categories = 13000)

test_fsq_results = res.json()

In [40]:
#Creating an empty list for number of venues within 1000 radius from each bike station
number_of_venues_list = []

#Creating an empty list to store the JSON responses for each location
foursquare_data = []

#Iterating through each row with lat & long in vancouver_bikes
for i, r in vancouver_bikes.iterrows():
    lat = r["latitude"]
    long = r["longitude"]
    
#Making a request for restaurants and bars as a venue category within a 1000-meter radius from a bike station
    res = get_venues_fsq(lat, long, radius = 1000, api_key = api_key_fsq, categories = 13000)
    foursquare_results = res.json()
    
#Getting the number of venues for each given location using the len function
    number_of_venues = len(foursquare_results["results"])

#Appending the number of venues for each bike location to a list
    number_of_venues_list.append(number_of_venues)

#Appending all results to a forsquare data dictionary
    for row in foursquare_results["results"]:
        foursquare_data.append({
            "fsq_id"           :row["fsq_id"],
            "distance_station" :row["distance"],
            "latitude"         :row["geocodes"]["main"]["latitude"],
            "longitude"        :row["geocodes"]["main"]["longitude"],
            "name"             :row["name"],
            "category_name"    :row["categories"][0]["name"],
            "rating"           :row.get("rating", None), #get() method used in case if there are empty values, so they can be replaced by None
            "city_bikes_id"    :r["uid"] 
            }) 

Put your parsed results into a DataFrame

In [41]:
venues_fsq = pd.DataFrame(foursquare_data)

In [42]:
venues_fsq.to_csv("vancouver_venues_fsq.csv", index = False)

In [43]:
venues_fsq

,fsq_id,distance_station,latitude,longitude,name,category_name,rating,city_bikes_id
0,4aec909ff964a520ccc821e3,435,49.263724,-123.108701,Milano Coffee,Coffee Shop,None,1
1,4aa98676f964a520105420e3,216,49.263077,-123.117243,Elysian Coffee Broadway,Coffee Shop,None,1
2,5a31538260d11b06bc47c475,527,49.263182,-123.107195,Modus Coffee,Café,None,1
3,4b1db335f964a520211424e3,97,49.263328,-123.114020,Marulilu Cafe,Café,None,1
4,5674aec9498e6800815c3598,556,49.257484,-123.114820,Vij's,Indian Restaurant,None,1
...,...,...,...,...,...,...,...,...
11946,4aa7f806f964a520524e20e3,736,49.276947,-123.126213,Two Parrots Taverna,Burger Joint,None,187
11947,4aa97278f964a520b25320e3,967,49.290242,-123.137571,Blenz Coffee,Coffee Shop,None,187
11948,4aa748faf964a5209a4c20e3,72,49.281472,-123.132602,Pumpjack Pub,Gay Bar,None,187
11949,4aa81ae3f964a520594f20e3,815,49.283983,-123.122190,Shabusen Yakiniku House,BBQ Joint,None,187


In [44]:
venues_fsq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11951 entries, 0 to 11950
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fsq_id            11951 non-null  object 
 1   distance_station  11951 non-null  int64  
 2   latitude          11951 non-null  float64
 3   longitude         11951 non-null  float64
 4   name              11951 non-null  object 
 5   category_name     11951 non-null  object 
 6   rating            0 non-null      object 
 7   city_bikes_id     11951 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 747.1+ KB


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [76]:
#Setting the api_key variable
# api_key_yelp = os.getenv("YELP_KEY")
api_key_yelp = "wRYebX_Z53j5k6SnHvp8VIMUxFUQDiqcapN3OPKnc6LIfsFO2CYluQ6gKg0YzkAhyzbThJ7AYZ60apdNW67A1rXokdT0K7A01JCmgpAJ_ai_SZH5AZgLzmuCOdi2ZnYx"

In [77]:
#Creating a function to make an API request to Yelp
def get_venues_yelp(latitude, longitude, radius, api_key, categories):
    url = "https://api.yelp.com/v3/businesses/search"
    
    headers = {
        "Accept": "application/json",
        "Authorization": "Bearer " + api_key
    }
    
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": radius,
        "categories": categories,
        "limit": 50
    }
    
    response = requests.get(url, headers=headers, params=params)
 
    return response


In [63]:
#Test request for the first row of vancouver_bikes df
res = get_venues_yelp(latitude = 49.262487, 
                      longitude = -123.114397, 
                      radius = 1000, 
                      api_key = api_key_yelp, 
                      categories = "restaurants")

test_yelp_results = res.json()

In [67]:
test_yelp_results["businesses"][0]["categories"][0]["alias"]

'japanese'

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [78]:
#Creating an empty list for number of venues within 1000 radius from each bike station
number_of_venues_list_yelp = []

#Creating an empty list to store the JSON responses for each location
yelp_data = []
        
for i, r in vancouver_bikes.iterrows():
    lat = r["latitude"]
    long = r["longitude"]
    
#Making a request for restaurants as a venue category within a 1000-meter radius from a bike station
    res = get_venues_yelp(lat, long, radius = 1000, api_key = api_key_yelp, categories = "restaurants")
    yelp_results = res.json()
    
#Getting the number of venues for each given location using the len function
    number_of_venues_yelp = len(yelp_results["businesses"])
    
#Appending the number of venues for each bike location to a list
    number_of_venues_list_yelp.append(number_of_venues_yelp)

#Appending all results to a yelp data dictionary
    for row in yelp_results["businesses"]:
        yelp_data.append({
            "yelp_id"          :row["id"],
            "name"             :row["name"],
            "rating"           :row["rating"],
            "category_name"    :row["categories"],
            "distance_station" :row["distance"],
            "latitude"         :row["coordinates"]["latitude"],
            "longitude"        :row["coordinates"]["longitude"],
            "city_bikes_id"    :r["uid"] 
            }) 

In [89]:
yelp_data[:1]

#I've made an error when getting "category_name" and to fix it and save API calls I will just iterate through the already gotten response to parse the category_name further

[{'yelp_id': 'XAH2HpuUUtu7CUO26pbs4w',
  'name': 'Saku',
  'rating': 4.3,
  'category_name': [{'alias': 'japanese', 'title': 'Japanese'}],
  'distance_station': 178.8453436692719,
  'latitude': 49.2631006,
  'longitude': -123.11667523250104,
  'city_bikes_id': 1}]

In [113]:
categories = []
for i in yelp_data:
    if i["category_name"]:  
        categories.append(i["category_name"][0]["alias"])
    else:
        categories.append(None) 

Put your parsed results into a DataFrame

In [114]:
venues_yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12425 entries, 0 to 12424
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   yelp_id           12425 non-null  object 
 1   name              12425 non-null  object 
 2   rating            12425 non-null  float64
 3   category_name     12425 non-null  object 
 4   distance_station  12425 non-null  float64
 5   latitude          12425 non-null  float64
 6   longitude         12425 non-null  float64
 7   city_bikes_id     12425 non-null  int64  
 8   category          12424 non-null  object 
dtypes: float64(4), int64(1), object(4)
memory usage: 873.8+ KB


In [115]:
venues_yelp

,yelp_id,name,rating,category_name,distance_station,latitude,longitude,city_bikes_id,category
0,XAH2HpuUUtu7CUO26pbs4w,Saku,4.3,"[{'alias': 'japanese', 'title': 'Japanese'}]",178.845344,49.263101,-123.116675,1,japanese
1,6iOAgzJ0DRZNSKA3FSrrOg,La Taqueria Pinche Taco Shop,4.2,"[{'alias': 'mexican', 'title': 'Mexican'}]",170.590723,49.263617,-123.112593,1,mexican
2,nkDZY5xqihF3XtZMzzfqqg,Hokkaido Ramen Santouka,4.2,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",191.044234,49.263127,-123.116892,1,noodles
3,4118Aq9LbkvUr4s719uUkA,Uma Sushi,4.3,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",152.157897,49.263805,-123.113729,1,japanese
4,F5wxgIiZE7LYQxgqhI483A,Seaport City Seafood Restaurant,4.2,"[{'alias': 'dimsum', 'title': 'Dim Sum'}, {'al...",168.615262,49.263921,-123.115152,1,dimsum
...,...,...,...,...,...,...,...,...,...
12420,RIb01mB9pEJ3yeebyHb4XA,Cioppino's Mediterranean Grill,4.2,"[{'alias': 'mediterranean', 'title': 'Mediterr...",1092.623823,49.275522,-123.121794,187,mediterranean
12421,hQoFgRJhYZn4AM5uE3c9fA,Donair Dude,3.7,"[{'alias': 'mideastern', 'title': 'Middle East...",76.317079,49.281217,-123.132758,187,mideastern
12422,VyHdjx6K9XNfjsh2_2PqFA,Thida Thai Resturant,3.3,"[{'alias': 'thai', 'title': 'Thai'}]",31.257034,49.281743,-123.133033,187,thai
12423,kDZTaeUaf78Jd25ktX8a8g,Number E Food,4.8,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",520.019263,49.277569,-123.131018,187,sandwiches


In [116]:
venues_yelp.to_csv("vancouver_venues_yelp.csv", index = False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

- Yelp appears to provide more data than Foursquare. For the same number of API calls (245 each for Foursquare and Yelp) and under the same conditions (limit of 50, radius of 1,000), I received 12,425 results from Yelp compared to 11,951 from Foursquare.
- Yelp's data is also more complete, as Foursquare's data sometimes has missing ratings.
- However, Foursquare offers better categorization, detailing the type of venue, while Yelp is less descriptive but more focused on specific cuisines.

Get the top 10 restaurants according to their rating

In [117]:
venues_yelp.sort_values("rating", ascending = False).head(10)

,yelp_id,name,rating,category_name,distance_station,latitude,longitude,city_bikes_id,category
11884,KGrEgFvTxJZEZ0GG7_kk9w,Seoul Hotdog,5.0,"[{'alias': 'hotdog', 'title': 'Hot Dogs'}, {'a...",96.151270,49.266496,-123.249855,2143,hotdog
12114,w4H0kLhdAtCBghZfL6rLkw,Firehouse Subs,5.0,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",616.123405,49.261070,-123.246391,2114,hotdogs
12091,KGrEgFvTxJZEZ0GG7_kk9w,Seoul Hotdog,5.0,"[{'alias': 'hotdog', 'title': 'Hot Dogs'}, {'a...",497.239589,49.266496,-123.249855,2114,hotdog
1741,6EkU5sUdWyxhNkKqHVtrGg,Tru Cafe,5.0,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",65.178204,49.269507,-123.138690,66,icecream
5896,1hP5JU-Pyo9BUUU7RwBgGQ,Kakai Sushi,5.0,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",119.150812,49.262189,-123.070825,242,sushi
11765,iPujVXzNJ-NYjhylcjSFxw,Domino's Pizza,5.0,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",254.880367,49.263665,-123.206821,715,pizza
11764,IW1m8-jcHuKdRKk4Pk3e8g,Chick Asta Grill,5.0,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",232.117676,49.264048,-123.207383,715,indpak
11246,KeeJMtxKKyxbWbe46BMlzw,Pretty Good Sandwiches,5.0,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",1058.985214,49.263310,-123.116310,524,sandwiches
10547,D-J3i1WCZxcdjixVSTrHCQ,Trim & Healthy,5.0,"[{'alias': 'delis', 'title': 'Delis'}]",591.163894,53.559010,-6.789490,480,delis
11746,WUON9ePviVw_vheP4BZSiw,Little Umbrella,5.0,"[{'alias': 'cafes', 'title': 'Cafes'}]",293.779225,49.263556,-123.205023,716,cafes
